In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from pathlib import Path
from datetime import datetime
import time
import getpass
import bs4
import os
import time
import requests

# 所有要建立的file 名稱 list
fileList = ['noon.txt', 'afternoon.txt', 'night.txt', 'person_leave.txt', 'other.txt', 'medical_treatment.txt']

# 定義 windowScroll 的 method
def windowScroll(driver, scrollToPos, n, sleepSec):
    for x in range(n):
    #為了符合臉書的動態頁面載入，讓scrollBar移動到 特定的位置(不是最下，這樣臉書載入時會出錯)
   # 這邊還要再細調整一下
        driver.execute_script(f"window.scrollTo(0,{scrollToPos})")
        print("scroll")
        time.sleep(sleepSec)
# 創建檔案
def makeFile(fileName):
    if not os.path.exists(fileName):
        Path(f"{fileName}").touch()
# 把資料寫入檔案中
def writeFile(fileName, text):
    f = open(fileName, 'a', encoding="UTF-8")
    f.write(text)
    f.close()
# 刪除單一file
def deleteFile(fileName):
    if os.path.exists(fileName):
        os.remove(fileName)
# 建立檔案的method (在這裡新增單一的makeFile)
def makeCollection():
    #創建中午班、下午班、醫療、請假 等 txt 檔案
    current = returnCurrentTime()
    for file in fileList:
        makeFile(file)
        writeFile(file, "【檔案建立時間: " + current + "】\n\n")
# 刪除之前匯出的所有txt file
def deleteCollection():
    for file in fileList:
        deleteFile(file)
# 取得現在時間
def returnCurrentTime():
    now = datetime.now()
    #print(now.strftime("%Y-%m-%d %H:%M"))
    return str(now.strftime("%Y-%m-%d %H:%M"))
# 在vscode 上輸入email & pwd
def enter_email_and_password():
    email =  driver.find_element_by_name('email')
    password = driver.find_element_by_name('pass')
    # 輸入user email
    emailString = getpass.getpass()
    email.send_keys(emailString)
    # 在teminal中輸入密碼(顯示為密碼字元 * )
    passwordString = getpass.getpass()
    password.send_keys(passwordString)
    password.submit()

def scrollCollection(time):
    windowScroll(driver, "document.body.scrollHeight*3/4", n= 10, sleepSec= 0.35)
    windowScroll(driver, "document.body.scrollHeight*10/11", n= 20, sleepSec= 0.5)
    windowScroll(driver, "document.body.scrollHeight*50/51", n= 20, sleepSec= 0.5)
    for i in range(time):
        windowScroll(driver, "document.body.scrollHeight*100/101", n= 40, sleepSec= 1)
#----------------主程式開始------------------
# 刪除之前匯出的所有txt file
deleteCollection()
#創建中午班、下午班、醫療、請假 的 txt 檔案
makeCollection()

# 社團連結(私人社團，需要登入帳號)
url = 'https://www.facebook.com/groups/198733930176314/?hoisted_section_header_type=recently_seen&multi_permalinks=4445181015531563'
# 關閉notification 直接進入網頁
chrome_options = webdriver.ChromeOptions()
# 跳過notification，讓selenium可以繼續執行下面的階段
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(chrome_options=chrome_options)
driver.get(url)
time.sleep(2)
driver.maximize_window()

# 用selenium 輸入email & 密碼
enter_email_and_password()
time.sleep(1.5)

#模擬滑鼠滾輪滾動(為了動態加載臉書社團頁面)
scrollCollection(3);
soup = bs4.BeautifulSoup(driver.page_source, 'html.parser')

#嘗試從比較大的層級去抓取人名 & 相對應的貼文內容
paras_computer = soup.findAll("div", class_ = "rq0escxv l9j0dhe7 du4w35lb hybvsw6c io0zqebd m5lcvass fbipl8qg nwvqtn77 k4urcfbm ni8dbmo4 stjgntxs sbcfpzgs")
for para in paras_computer:
    # 紀錄當前 para 要印出的所有訊息
    tempString = ""
    name = para.find("strong")
    if name != None:
        print(name.span.text)
        tempString += ("發文者: " + name.span.text + "\n")
    # 電腦發文的text
    computer_texts = para.find("div", class_="kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x c1et5uql ii04i59q")
    # 有包含這個元素才做以下動作 (手機板發的文不包含以上的class)
    if computer_texts != None:
        texts_all = computer_texts.findAll("div", dir= "auto")
        for text in texts_all:
            print(text.text)
            tempString += (text.text + "\n")

    # 手機發文的class
    phone_texts = para.find("div", class_="kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x c1et5uql")
    if phone_texts != None:
        print(phone_texts.text)
        tempString += (phone_texts.text)
    print("\n")
    # 根據當前文章的關鍵字做分類
    for i in range(len(tempString)):
        if tempString[i:i+2] == "請假":
            writeFile("personal_leave.txt", tempString+"\n------------\n")
            break
        if tempString[i:i+2] == "醫療":
            writeFile("medical_treatment.txt", tempString+"\n------------\n")
            break
        if tempString[i:i+2] == "醫院":
            writeFile("medical_treatment.txt", tempString+"\n------------\n")
            break
        if tempString[i:i+2] == "中午":
            writeFile("noon.txt", tempString+"\n------------\n")
            break        
        if tempString[i:i+2] == "下午":
            writeFile("afternoon.txt", tempString+"\n------------\n")
            break
        if tempString[i:i+2] == "環校":
            writeFile("night.txt", tempString+"\n------------\n")
            break
        if i == len(tempString)-1:
            writeFile("other.txt", tempString+"\n-------------\n")
    tempString = ""

  


<ipython-input-7-8ec63e01a0a7>:79: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options)
<ipython-input-7-8ec63e01a0a7>:56: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  email =  driver.find_element_by_name('email')
<ipython-input-7-8ec63e01a0a7>:57: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  password = driver.find_element_by_name('pass')


scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll
scroll


林泓宇
【一月假日班表】
一月的班表出來囉~一般來說排班組應該要滿三個月排假日班，期末才有時數。這個月(十二月)過完後會在班表更新來的月份數，還沒滿的快來跟我談補班喔~


林